In [25]:
def plot_time_serie(data, vars_):

  import plotly.graph_objects as go

  fig = go.Figure()

  for v in vars_:
    fig.add_trace(go.Scatter(x=data[v].index, y=data[v],
                    mode='lines',
                    name=v))

  fig.show()

In [26]:
def plot_yy(y_mod,y_mes):

  import plotly.graph_objects as go

  fig = go.Figure()

 
  fig.add_trace(go.Scatter(x=y_mod, y=y_mes,
                    mode='markers',
                    name='yyplot'))

  fig.show()

In [17]:
def MAPE(y_mesuree, y_modelisee):
  # Erreur moyenne relative des modèles (MAPE)
  import numpy as np
  diff = (y_modelisee - y_mesuree)/y_mesuree
  mape = np.abs(diff).sum()/len(diff)*100
  return np.round(mape,1)

In [2]:
File_power= "/content/drive/My Drive/Reseauchaleur/Data/sst/SS_DHN_Power_kW.pkl"
import pandas as pd
df_sst_power = pd.read_pickle(File_power)
df_sst_power['totale_demand'] = df_sst_power.sum(axis=1)
df_sst_power_H = df_sst_power.resample('H').mean()

In [ ]:
plot_time_serie(df_sst_power_H,['totale_demand'])

In [4]:
File_meteo = "/content/drive/My Drive/Reseauchaleur/Data/meteo_varna.csv"
dateparse = lambda x: pd.datetime.strptime(x, '%m/%d/%Y %H:%M:%S')
df_meteo  = pd.read_csv(File_meteo,sep=',',decimal=".",parse_dates=['dateHrLwt'],date_parser=dateparse,low_memory = False,na_values=' ')
df_meteo.set_index('dateHrLwt', inplace=True)

df_meteo = df_meteo[['windChillTemperatureCelsius','apparentTemperatureCelsius']]
data = df_sst_power_H.merge(df_meteo, how='outer', left_index = True, right_index = True)
data =  data[['totale_demand', 'windChillTemperatureCelsius','apparentTemperatureCelsius']]
data = data[((data.totale_demand - data.totale_demand.mean()) / data.totale_demand.std()).abs() < 3]

data.dropna(inplace=True)
data['hour_of_day'] = data.index.map(lambda x: str(x.hour)+'H')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  


In [5]:
data.head()

,totale_demand,windChillTemperatureCelsius,apparentTemperatureCelsius,hour_of_day
2015-09-01 03:00:00,192.200000,19.7,19.7,3H
2015-09-01 04:00:00,199.672857,19.1,19.1,4H
2015-09-01 05:00:00,228.255000,19.0,19.0,5H
2015-09-01 06:00:00,206.688750,18.9,18.9,6H
2015-09-01 07:00:00,360.351667,19.5,19.5,7H


In [6]:
from sklearn.model_selection import train_test_split

data_windows = data[(data.index>='2015-11-01 00:00:00') & (data.index<='2016-04-01 00:00:00')	]

var_selec = ['apparentTemperatureCelsius', 'hour_of_day']

X = data_windows[var_selec].values
y = data_windows['totale_demand'].values

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25)
# 
print('Train', X_train.shape, y_train.shape)
print('Test', X_test.shape, y_test.shape)

Train (2546, 2) (2546,)
Test (849, 2) (849,)


In [7]:
plot_time_serie(data_windows,['totale_demand'])

# Regression linéaire simple

In [16]:
data_wind_dum = pd.get_dummies(data_windows)
col_x = data_wind_dum.columns.to_list()
col_x.remove('totale_demand')
col_x.remove('apparentTemperatureCelsius')


X = data_wind_dum[col_x].values
y = data_wind_dum['totale_demand'].values

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25)


In [19]:
from sklearn.linear_model import LinearRegression


linreg = LinearRegression().fit(X_train, y_train)
Ypredit_test = linreg.predict(X_test)

print('R2 score(training): {:.3f}'
     .format(linreg.score(X_train, y_train)))
print('R2 score (test): {:.3f}'
     .format(linreg.score(X_test, y_test)))
print('MAPE (%): ',MAPE(y_test,Ypredit_test))



data_wind_dum['totale_demand_predicted'] = linreg.predict(X)

plot_time_serie(data_wind_dum,['totale_demand','totale_demand_predicted'])

R2 score(training): 0.513
R2 score (test): 0.503
MAPE (%):  75.3


In [21]:
from sklearn.linear_model import Lasso

data_wind_dum = pd.get_dummies(data_windows)
col_x = data_wind_dum.columns.to_list()
col_x.remove('totale_demand')
col_x.remove('apparentTemperatureCelsius')


X = data_wind_dum[col_x].values
y = data_wind_dum['totale_demand'].values

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25)

for alpha in [0.01,0.05,0.1,0.2,0.3,0.4,0.5, 1, 2, 3]:
    linlasso = Lasso(alpha, max_iter = 10000).fit(X_train, y_train)
    r2_train = linlasso.score(X_train, y_train)
    r2_test = linlasso.score(X_test, y_test)
    print(r2_train, r2_test)
    Ypredit_test = linlasso.predict(X_test)

0.4964661741760401 0.5533748715895157
0.49646165319465085 0.553318190026574
0.4964481535081772 0.5532114665630542
0.49639576029112054 0.5529874190286559
0.49631321763697206 0.5527464327229055
0.496198480766432 0.5524740399670691
0.49605102410122254 0.5521695890843843
0.49492090575137354 0.5499614424704196
0.49067441199007195 0.5441304296839122
0.48377132212960056 0.535729960968358


# Arbre de decision

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(X_train, y_train)
regr_2.fit(X_train, y_train)


y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)


In [ ]:
print(regr_2.score(X_train,y_train), regr_2.score(X_test,y_test))

0.7576314539544533 0.7059722884073759


In [ ]:
X = data[var_selec].values
y = data['totale_demand'].values

data['totale_demand_predicted'] = regr_2.predict(X)


In [ ]:
plot_time_serie(data,['totale_demand','totale_demand_predicted'])

In [31]:
from sklearn.ensemble import GradientBoostingRegressor

data['hour_of_day'] = data.index.map(lambda x: x.hour)

data_windows = data[(data.index>='2015-12-12 00:00:00') & (data.index<='2016-02-12 00:00:00')	]



var_selec = ['apparentTemperatureCelsius', 'hour_of_day']

X = data_windows[var_selec].values
y = data_windows['totale_demand'].values



X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.25)

reg = GradientBoostingRegressor(n_estimators=200)
reg.fit(X_train, y_train)
Ypredit_test = reg.predict(X_test)
r2_train = reg.score(X_train, y_train)
r2_test  = reg.score(X_test, y_test)
print(r2_train, r2_test)


data_windows['totale_demand_predicted'] = reg.predict(X)

plot_time_serie(data_windows,['totale_demand','totale_demand_predicted'])

0.6907589007053128 0.4052183344488777


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
plot_yy(Ypredit_test,y_train)
